In [ ]:
!pip install python-pptx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 14.2 MB/s eta 0:00:00
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470935 sha256=889d93e1e8d738260ed6e9c1f0cc3fed8fda4080ba6be7008966f4d5360f4ab3
  Stored in directory: /root/.cache/pip/wheels/ea/dd/74/01b3ec7256a0800b99384e9a0f7620e358afc3a51a59bf9b49
Successfully built python-pptx


In [ ]:
from pptx import Presentation
import pandas as pd
import re
import os

In [ ]:
!rm -rf 321slides
!rm -rf __MACOSX
!unzip 321slides.zip -d /content
!ls 321slides

Archive:  321slides.zip
   creating: /content/321slides/
  inflating: /content/__MACOSX/._321slides  
  inflating: /content/321slides/13-concurrency.pptx  
  inflating: /content/__MACOSX/321slides/._13-concurrency.pptx  
  inflating: /content/321slides/11-unixio.pptx  
  inflating: /content/__MACOSX/321slides/._11-unixio.pptx  
  inflating: /content/321slides/09-memory.pptx  
  inflating: /content/__MACOSX/321slides/._09-memory.pptx  
  inflating: /content/321slides/07-linking.pptx  
  inflating: /content/__MACOSX/321slides/._07-linking.pptx  
  inflating: /content/321slides/10-vm.pptx  
  inflating: /content/__MACOSX/321slides/._10-vm.pptx  
  inflating: /content/321slides/06-assembly.pptx  
  inflating: /content/__MACOSX/321slides/._06-assembly.pptx  
  inflating: /content/321slides/12-networking.pptx  
  inflating: /content/__MACOSX/321slides/._12-networking.pptx  
  inflating: /content/321slides/02-simple-data.pptx  
  inflating: /content/__MACOSX/321slides/._02-simple-data.pptx  


In [ ]:
def process_pptx_files(filepath_list):

    file_dict = {}

    for file in filepath_list:
        f = open(f'{file}', "rb")
        pres = Presentation(f)

        for i in range(len(pres.slides)):
            text_runs = []
            for shape in pres.slides[i].shapes:
                if shape.has_text_frame:
                    for paragraph in shape.text_frame.paragraphs:
                        for run in paragraph.runs:
                            text_runs.append(run.text)

            file_dict[(file + "_" + str(i))] = " ".join(text_runs)

    return file_dict

In [ ]:
pptx_paths = os.listdir("321slides")
pptx_paths = ["321slides/" + s for s in pptx_paths]
print(pptx_paths)

pptx_dict = {}
pptx_dict.update(process_pptx_files(pptx_paths))

['321slides/01-overview.pptx', '321slides/06-assembly.pptx', '321slides/07-linking.pptx', '321slides/11-unixio.pptx', '321slides/12-networking.pptx', '321slides/05-allocation.pptx', '321slides/03-arrays-pointers.pptx', '321slides/08-exceptions.pptx', '321slides/13-concurrency.pptx', '321slides/02-simple-data.pptx', '321slides/10-vm.pptx', '321slides/04-struct-union.pptx', '321slides/09-memory.pptx']


In [ ]:
df_pptx = pd.DataFrame([pptx_dict]).T.reset_index()
df_pptx.columns = ["file_name", "text"]

df_pptx_clean = df_pptx[df_pptx["text"].str.len() > 0].reset_index(drop=True)
df_pptx_clean["text_clean"] = df_pptx_clean["text"].apply(lambda x : re.sub(r'[^a-zA-Z0-9_.\s]+', '', str(x)))
df_pptx_clean.tail()

,file_name,text,text_clean
727,321slides/09-memory.pptx_100,Cox Dynamic Memory Allocation Compaction Movin...,Cox Dynamic Memory Allocation Compaction Movin...
728,321slides/09-memory.pptx_101,Cox Dynamic Memory Allocation GC Variations Ma...,Cox Dynamic Memory Allocation GC Variations Ma...
729,321slides/09-memory.pptx_102,Cox Dynamic Memory Allocation Conservative GC ...,Cox Dynamic Memory Allocation Conservative GC ...
730,321slides/09-memory.pptx_103,Cox Dynamic Memory Allocation GC vs. malloc /...,Cox Dynamic Memory Allocation GC vs. malloc ...
731,321slides/09-memory.pptx_104,Cox Dynamic Memory Allocation Next Time Virtua...,Cox Dynamic Memory Allocation Next Time Virtua...


In [ ]:
# Add the new column with values from 0 to n-1
df_pptx_clean['DOC_ID'] = range(len(df_pptx_clean))

# Save the updated DataFrame back to the CSV file
df_pptx_clean.to_csv('/content/pptx_data.csv', index=False)